In [1]:
import argparse
import json
import os
import sys

import apex
import numpy as np
import torch
import yaml
from det3d import __version__, torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.datasets.kitti import kitti_common as kitti
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
)
from det3d.torchie.trainer import get_dist_info, load_checkpoint
from det3d.torchie.trainer.utils import all_gather, synchronize
from torch.nn.parallel import DistributedDataParallel

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description="Train a detector")
    parser.add_argument("config", help="train config file path")
    parser.add_argument("--work_dir", help="the dir to save logs and models")
    parser.add_argument(
        "--checkpoint", help="the dir to checkpoint which the model read from"
    )
    parser.add_argument(
        "--txt_result",
        type=bool,
        default=False,
        help="whether to save results to standard KITTI format of txt type",
    )
    parser.add_argument(
        "--gpus",
        type=int,
        default=1,
        help="number of gpus to use " "(only applicable to non-distributed training)",
    )
    parser.add_argument(
        "--launcher",
        choices=["none", "pytorch", "slurm", "mpi"],
        default="none",
        help="job launcher",
    )
    parser.add_argument("--local_rank", type=int, default=0)

    #args = parser.parse_args() #elodie

    #args = parser.parse_args('examples/second/configs/kitti_all_vfev3_spmiddlefhd_rpn1_mghead_syncbn.py --work_dir /home/elodie/det3D_Output/Outputs/Det3D_Outputs/SECOND_all_3_20200220-101802 --checkpoint /home/elodie/det3D_Output/Outputs/Det3D_Outputs/SECOND_all_3_20200220-101802/latest.pth --txt_result True'.split())
    args = parser.parse_args('examples/second/configs/kitti_all_vfev3_spmiddlefhd_rpn1_mghead_syncbn_freatures3.py --work_dir /home/elodie/model_pth/SECOND_F3_all_1_20200305-093312 --checkpoint /home/elodie/model_pth/SECOND_F3_all_1_20200305-093312/latest.pth --txt_result True'.split())

    if "LOCAL_RANK" not in os.environ:
        os.environ["LOCAL_RANK"] = str(args.local_rank)

    return args

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
args = parse_args()
cfg = Config.fromfile(args.config)
cfg.local_rank = args.local_rank
# update configs according to CLI args
if args.work_dir is not None:
    cfg.work_dir = args.work_dir
#print("Config:\n cfg.local_rank:",cfg.local_rank,"\ncfg.data.val:",cfg.data.val)

In [4]:
distributed = False
if "WORLD_SIZE" in os.environ:
    distributed = int(os.environ["WORLD_SIZE"]) > 1
print("distributed:",distributed)
if distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend="nccl", init_method="env://")

    cfg.gpus = torch.distributed.get_world_size()
else:
    cfg.gpus = args.gpus
print("cfg.gpus:",cfg.gpus)
logger = get_root_logger(cfg.log_level)

distributed: False
cfg.gpus: 1


In [5]:
dataset = build_dataset(cfg.data.val)
data_loader = build_dataloader(
    dataset,
    batch_size=cfg.data.samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=False,
)
print("data_loader:",data_loader)

data_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f5d8c364390>


In [6]:
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

2020-03-28 18:09:25,895 - INFO - Finish RPN Initialization
2020-03-28 18:09:25,896 - INFO - num_classes: [1, 1, 1], num_preds: [14, 14, 14], num_dirs: [4, 4, 4]
2020-03-28 18:09:25,898 - INFO - Finish MultiGroupHead Initialization


In [7]:
checkpoint = load_checkpoint(model, args.checkpoint, map_location="cpu")

In [8]:
model = model.cuda()#cuda()函数都能实现从CPU到GPU的内存迁移,对model自身进行的内存迁移。

In [9]:
model.eval()

VoxelNet(
  (reader): VoxelFeatureExtractorV3()
  (backbone): SpMiddleFHD(
    (middle_conv): SparseSequential(
      (0): SubMConv3d()
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): SubMConv3d()
      (4): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): SparseConv3d()
      (7): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): SubMConv3d()
      (10): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (11): ReLU()
      (12): SubMConv3d()
      (13): BatchNorm1d(32, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): SparseConv3d()
      (16): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (17): ReLU()
      (18): SubMConv3d()
      (19): BatchNorm1d(64, eps=0.001, momentum

In [10]:
print(cfg.local_rank)
prog_bar = torchie.ProgressBar(len(data_loader.dataset) // cfg.gpus)

0
[                              ] 0/3769, elapsed: 0s, ETA:

In [11]:
detections = {}
cpu_device = torch.device("cpu")

In [12]:
# aa = 0
for i, data_batch in enumerate(data_loader):
#     aa = aa+1
#     if aa == 2:
#         break
    with torch.no_grad():  #torch.no_grad()禁用对autograd中的梯度的跟踪
        print("torch.no_grad()")
#         print(data_batch)
        outputs = batch_processor(
            model, data_batch, train_mode=False, local_rank=args.local_rank,
        )
#         print(outputs)
    for output in outputs:
        token = output["metadata"]["token"]
        for k, v in output.items():
            if k not in [
                "metadata",
            ]:
                output[k] = v.to(cpu_device)
        detections.update(
            {token: output,}
        )
        if args.local_rank == 0:
            prog_bar.update()


torch.no_grad()
[                              ] 6/3769, 1.7 task/s, elapsed: 3s, ETA:  2162storch.no_grad()
[                              ] 12/3769, 3.2 task/s, elapsed: 4s, ETA:  1184storch.no_grad()
[                              ] 18/3769, 4.5 task/s, elapsed: 4s, ETA:   842storch.no_grad()
[                              ] 24/3769, 5.6 task/s, elapsed: 4s, ETA:   664storch.no_grad()
[                              ] 30/3769, 6.6 task/s, elapsed: 5s, ETA:   563storch.no_grad()
[                              ] 36/3769, 7.6 task/s, elapsed: 5s, ETA:   493storch.no_grad()
[                              ] 42/3769, 8.5 task/s, elapsed: 5s, ETA:   440storch.no_grad()
[                              ] 48/3769, 9.1 task/s, elapsed: 5s, ETA:   407storch.no_grad()
[                              ] 54/3769, 9.8 task/s, elapsed: 5s, ETA:   378storch.no_grad()
[                              ] 60/3769, 10.5 task/s, elapsed: 6s, ETA:   353storch.no_grad()
[                              ] 66/3769, 11

[>>>>                          ] 510/3769, 18.2 task/s, elapsed: 28s, ETA:   179storch.no_grad()
[>>>>                          ] 516/3769, 18.2 task/s, elapsed: 28s, ETA:   179storch.no_grad()
[>>>>                          ] 522/3769, 18.2 task/s, elapsed: 29s, ETA:   178storch.no_grad()
[>>>>                          ] 528/3769, 18.3 task/s, elapsed: 29s, ETA:   177storch.no_grad()
[>>>>                          ] 534/3769, 18.4 task/s, elapsed: 29s, ETA:   176storch.no_grad()
[>>>>                          ] 540/3769, 18.4 task/s, elapsed: 29s, ETA:   176storch.no_grad()
[>>>>                          ] 546/3769, 18.3 task/s, elapsed: 30s, ETA:   176storch.no_grad()
[>>>>                          ] 552/3769, 18.3 task/s, elapsed: 30s, ETA:   176storch.no_grad()
[>>>>                          ] 558/3769, 18.4 task/s, elapsed: 30s, ETA:   175storch.no_grad()
[>>>>                          ] 564/3769, 18.4 task/s, elapsed: 31s, ETA:   174storch.no_grad()
[>>>>                         

KeyboardInterrupt: 

In [ ]:
print(len(detections))

In [ ]:
synchronize()
all_predictions = all_gather(detections)
predictions = {}
for p in all_predictions:
    predictions.update(p)

In [ ]:
result_dict, dt_annos = dataset.evaluation(predictions, output_dir=args.work_dir)

# result_dict, detections = test(
#    data_loader, model, save_dir=None, distributed=distributed
#    )
print(detections)

In [ ]:
print(dt_annos[3701])

In [ ]:
if args.txt_result:
    res_dir = "/home/elodie/DockerDet3D/predictions_xyz_0"
    # for k, dt in predictions.items():
    #for k, dt in dt_annos.items(): #elodie
    for i in range(len(dt_annos)): #elodie
        dt = dt_annos[i]
        with open(
            os.path.join(res_dir, "%06d.txt" % int(dt["metadata"]["token"])), "w"
        ) as fout:
            print(dt)
            lines = kitti.annos_to_kitti_label(dt)
            for line in lines:
                fout.write(line + "\n")

#     ap_result_str, ap_dict = kitti_evaluate(
#         "/data/Datasets/KITTI/Kitti/object/training/label_2",
#         res_dir,
#         label_split_file="/data/Datasets/KITTI/Kitti/ImageSets/val.txt",
#         current_class=0,
#     )

#     print(ap_result_str)